## Introduction 

The script scrapes data from the https://thehackernews.com/ website

## Packages

In [1]:
#import packages 
import pandas as pd 
import numpy as np
import os 
import itertools

#import time related packages
import time
from datetime import date
from datetime import datetime
from datetime import timedelta


#import webscraping packages 
import bs4 as bs
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import chromedriver_autoinstaller
from selenium_stealth import stealth
import undetected_chromedriver as uc

#postgresql database packages 
import psycopg2
import psycopg2.extras as extras
from sqlalchemy import create_engine

import csv
from io import StringIO

## Mine the data

In [2]:
#create an empty dataframe
df = pd.DataFrame()
content_df = pd.DataFrame()

#get the url 
url = 'https://thehackernews.com/'

driver = uc.Chrome(use_subprocess=True)
driver.get(url)
driver.maximize_window()
time.sleep(5)

#generate a soup object
soup = bs.BeautifulSoup(driver.page_source,'lxml')


WebDriverException: Message: unknown error: cannot connect to chrome at 127.0.0.1:59179
from session not created: This version of ChromeDriver only supports Chrome version 105
Current browser version is 104.0.5112.102
Stacktrace:
Backtrace:
	Ordinal0 [0x0059DF13+2219795]
	Ordinal0 [0x00532841+1779777]
	Ordinal0 [0x0044423D+803389]
	Ordinal0 [0x004664AC+943276]
	Ordinal0 [0x0045F2B4+914100]
	Ordinal0 [0x0045F091+913553]
	Ordinal0 [0x004936B9+1128121]
	Ordinal0 [0x0049331A+1127194]
	Ordinal0 [0x0048E616+1107478]
	Ordinal0 [0x00467F89+950153]
	Ordinal0 [0x00468F56+954198]
	GetHandleVerifier [0x00892CB2+3040210]
	GetHandleVerifier [0x00882BB4+2974420]
	GetHandleVerifier [0x00636A0A+565546]
	GetHandleVerifier [0x00635680+560544]
	Ordinal0 [0x00539A5C+1808988]
	Ordinal0 [0x0053E3A8+1827752]
	Ordinal0 [0x0053E495+1827989]
	Ordinal0 [0x005480A4+1867940]
	BaseThreadInitThunk [0x767AFA29+25]
	RtlGetAppContainerNamedObjectPath [0x77007B5E+286]
	RtlGetAppContainerNamedObjectPath [0x77007B2E+238]


In [3]:
#get the main 
main_df = pd.DataFrame()

main_tag = soup.find_all('div', {'class': 'body-post clear'})
# main_tag

for i in main_tag:
    
    #get the data 
    main_header = i.find_all('h2')
    main_header = [t.text.strip() for t in main_header][0]
    
    #get the dates 
    main_dates = i.find_all('div', class_ = 'item-label')
    main_published_dates = [y.text.rstrip().split('\ue804')[0].replace('\ue802', '') for y in main_dates][0]

    #get the author of the artile
    main_author = [x.text.rstrip().split('\ue804')[1] for x in main_dates][0]
    
    #get the summary of the details 
    main_sum_p = i.find_all('div', {'class': 'home-desc'})
    main_sum_par = [z.text.rstrip().replace('\xa0', ' ').replace('\\', '') for z in main_sum_p][0]
    
    #get the urls to full story 
    main_full_story_url = i.find_all('a', href = True) 
    main_full_story_url_url = [x['href'] for x in main_full_story_url]
#     print(main_full_story_url_url)
    
    #get convert the lists into a dataframe
    main_extracted_df = {'topics':main_header, 'dates':main_published_dates, 'author':main_author, 'links':main_full_story_url_url,'truncated content':main_sum_par}
    
    #turn the data into a dataframe
    main_raw_df = pd.DataFrame(main_extracted_df)

    #append the data into created dataframe
    main_df = pd.concat([main_df, main_raw_df])
    
    
#reset the index
main_df.reset_index(drop = True, inplace = True)
main_df['page'] = 'main page'

#print the data
main_df.head(5)

,topics,dates,author,links,truncated content,page
0,Hackers Breach LastPass Developer System to St...,"August 26, 2022",Ravie Lakshmanan,https://thehackernews.com/2022/08/hackers-brea...,Password management service LastPass confirme...,main page
1,Cybercrime Groups Increasingly Adopting Sliver...,"August 26, 2022",Ravie Lakshmanan,https://thehackernews.com/2022/08/cybercrime-g...,Nation-state threat actors are increasingly ...,main page
2,Okta Hackers Behind Twilio and Cloudflare Brea...,"August 25, 2022",Ravie Lakshmanan,https://thehackernews.com/2022/08/okta-hackers...,The threat actor behind the attacks on Twili...,main page
3,Microsoft Uncovers New Post-Compromise Malware...,"August 25, 2022",Ravie Lakshmanan,https://thehackernews.com/2022/08/microsoft-un...,The threat actor behind the SolarWinds supply...,main page
4,U.S. Government Spending Billions on Cybersecu...,"August 25, 2022",The Hacker News,https://thehackernews.com/2022/08/us-governmen...,"In recent months, the House of Representative...",main page


In [4]:
driver.find_element(By.XPATH,' //*[@id="Blog1_blog-pager-older-link"]').click()

In [5]:
#generate a soup object
# soup_test = bs.BeautifulSoup(driver.page_source,'lxml')

In [6]:
# soup

In [7]:
# soup_test

In [8]:
#create an empty dataframe
df = pd.DataFrame()
content_df = pd.DataFrame()

#get the url 
url = 'https://thehackernews.com/'

driver = uc.Chrome(use_subprocess=True)
driver.get(url)
driver.maximize_window()
time.sleep(5)

#generate a soup object
soup = bs.BeautifulSoup(driver.page_source,'lxml')

tags = soup.find_all('div', {'class': 'body-post clear'})


#get the headers links 
headers = soup.find_all('div', {'class': 'menu-box cf'})
#create the place holder for links
links = []

#loop through the headers categories
for i in headers:
    row = i.find_all('li')
    row = i.find_all('a', href = True)    
    new_url = [x['href'] for x in row]
    links.append(new_url)

    #get the url
    for x in links[0][1:5]:
        base_url = url+x
        page_focus = base_url.split('/')[-1].replace('%20', ' ')
        print(f"The page: {page_focus} url: {base_url}")

        #get the  page data
        driver = uc.Chrome(use_subprocess=True)
        driver.get(base_url)
        driver.maximize_window()
        time.sleep(5)
        soup_headers = bs.BeautifulSoup(driver.page_source,'lxml')

        #get the headers 
        header_tags = soup_headers.find_all('div', {'class', 'body-post clear'})

        #loop through the page and get the data
        for i in header_tags:

            #get the urls to full story 
            full_story_url = i.find_all('a', href = True) 
            full_story_url = [x['href'] for x in full_story_url]

            #get the topics 
            topics = i.find_all('h2')
            topics = [x.text for x in topics]

            #get the dates 
            dates = i.find_all('div', class_ = 'item-label')
            published_dates = [y.text.rstrip().split('\ue804')[0].replace('\ue802', '') for y in dates]

            #get the author of the artile
            author = [x.text.rstrip().split('\ue804')[1] for x in dates]

            #get the summary of the details 
            sum_p = i.find_all('div', {'class': 'home-desc'})
            sum_par = [z.text.rstrip().replace('\xa0', ' ').replace('\\', '') for z in sum_p]

                        
            #get convert the lists into a dataframe
            extracted_df = {'topics':topics, 'dates':published_dates, 'author':author, 'links':full_story_url,
                           'truncated content':sum_par}

            #turn the data into a dataframe
            raw_df = pd.DataFrame(extracted_df)
            raw_df['page'] = page_focus

            #append the data into created dataframe
            df = pd.concat([df, raw_df])
    
 
# #print the extacted data
df.reset_index(drop = True, inplace = True)

#print a sample of the data 
df.sample(5)

The page: data breach url: https://thehackernews.com//search/label/data%20breach
The page: Cyber Attack url: https://thehackernews.com//search/label/Cyber%20Attack
The page: Vulnerability url: https://thehackernews.com//search/label/Vulnerability
The page: Malware url: https://thehackernews.com//search/label/Malware


,topics,dates,author,links,truncated content,page
29,Meta Cracks Down on Cyber Espionage Operations...,"August 08, 2022",Ravie Lakshmanan,https://thehackernews.com/2022/08/meta-cracks-...,Facebook parent company Meta disclosed that i...,Cyber Attack
25,Conti Cybercrime Cartel Using 'BazarCall' Phis...,"August 11, 2022",Ravie Lakshmanan,https://thehackernews.com/2022/08/conti-cyberc...,A trio of offshoots from the notorious Conti ...,Cyber Attack
38,CISA Adds 7 New Actively Exploited Vulnerabili...,"August 20, 2022",Ravie Lakshmanan,https://thehackernews.com/2022/08/cisa-adds-7-...,The U.S. Cybersecurity and Infrastructure Sec...,Vulnerability
17,France Rules That Using Google Analytics Viola...,"February 11, 2022",Ravie Lakshmanan,https://thehackernews.com/2022/02/france-rules...,French data protection regulators on Thursday...,data breach
10,Block Admits Data Breach Involving Cash App Da...,"April 06, 2022",Ravie Lakshmanan,https://thehackernews.com/2022/04/block-admits...,"Block, the company formerly known as Square, ...",data breach


In [9]:
overal_df = pd.concat([main_df, df])
overal_df.reset_index(drop = True, inplace = True)
overal_df.sample(5)

,topics,dates,author,links,truncated content,page
56,SonicWall Issues Patch for Critical Bug Affect...,"July 23, 2022",Ravie Lakshmanan,https://thehackernews.com/2022/07/sonicwall-is...,Network security company SonicWall on Friday ...,Vulnerability
0,Hackers Breach LastPass Developer System to St...,"August 26, 2022",Ravie Lakshmanan,https://thehackernews.com/2022/08/hackers-brea...,Password management service LastPass confirme...,main page
59,Microsoft Releases Fix for Zero-Day Flaw in Ju...,"July 13, 2022",Ravie Lakshmanan,https://thehackernews.com/2022/07/microsoft-re...,Microsoft released its monthly round of Patch...,Vulnerability
63,Hackers Using Fake DDoS Protection Pages to Di...,"August 24, 2022",Ravie Lakshmanan,https://thehackernews.com/2022/08/hackers-usin...,WordPress sites are being hacked to display f...,Malware
60,Hackers Exploiting Follina Bug to Deploy Rozen...,"July 09, 2022",Ravie Lakshmanan,https://thehackernews.com/2022/07/hackers-expl...,A newly observed phishing campaign is leverag...,Vulnerability


In [10]:
print(overal_df['links'][:3])

0    https://thehackernews.com/2022/08/hackers-brea...
1    https://thehackernews.com/2022/08/cybercrime-g...
2    https://thehackernews.com/2022/08/okta-hackers...
Name: links, dtype: object


In [11]:
#create a content df 
con_df = pd.DataFrame()
count = 0
    
#get the url 
for link in  overal_df['links']:
    count +=1
    print(f"Website no: {count} link: {link}")
    
    #load the page
    driver = uc.Chrome(use_subprocess=True)
    driver.get(link)
    time.sleep(2)

    #generate a soup object
    content_soup = bs.BeautifulSoup(driver.page_source,'lxml')
    
    
    for i in content_soup.find_all('div', {'class':'main-box clear'}):

        #get the header
        hd = i.find_all('h1')
        header = [x.text.strip() for x in hd][0]
        print(f" {'*'*5} story: {header} {'*'*5}")

        #get the url 
        con_ls_df = {"content":[]}
        links_ls_df = {"links":[]}
        for p in i.find_all('p'):
            con_ls = [x.text.strip() for x in p][0]
            con_ls_df['content'].append(con_ls)
            
            #links 
            ik = p.find_all('a')
            extra_links = [x['href'].strip() for x in ik]
            
            for lp in extra_links:
                links_ls_df['links'].append(lp)
            
           

        #get con frames 
        con_frames = {'topics':header, 'full story':con_ls_df, "extra links":links_ls_df}
        con_frames_df = pd.DataFrame(con_frames)
        con_df = pd.concat([con_df, con_frames_df])

        
        
#clen the data 
con_df_1 = con_df[con_df['extra links'].isna()==True]
con_df_1.drop(columns = ['extra links'], inplace = True, axis = 1)

con_df_2 = con_df[con_df['extra links'].isna()==False]
con_df_2.drop(columns = ['full story'], inplace = True, axis = 1)

con_df = pd.merge(con_df_1, con_df_2, how = 'left', on = 'topics')
con_df['full story'] = con_df['full story'].apply(lambda x: "".join(x))
con_df['extra links'] = con_df['extra links'].apply(lambda x: "; ".join(x))

# reset index 
con_df.reset_index(drop = True, inplace = True)

con_df

Website no: 1 link: https://thehackernews.com/2022/08/hackers-breach-lastpass-developer.html
 ***** story: Hackers Breach LastPass Developer System to Steal Source Code *****
Website no: 2 link: https://thehackernews.com/2022/08/cybercrime-groups-increasingly-adopting.html
 ***** story: Cybercrime Groups Increasingly Adopting Sliver Command-and-Control Framework *****
Website no: 3 link: https://thehackernews.com/2022/08/okta-hackers-behind-twilio-and.html
 ***** story: Okta Hackers Behind Twilio and Cloudflare Breach Hit Over 130 Organizations *****
Website no: 4 link: https://thehackernews.com/2022/08/microsoft-uncovers-new-post-compromise.html
 ***** story: Microsoft Uncovers New Post-Compromise Malware Used by Nobelium Hackers *****
Website no: 5 link: https://thehackernews.com/2022/08/us-government-spending-billions-on.html
 ***** story: U.S. Government Spending Billions on Cybersecurity *****
Website no: 6 link: https://thehackernews.com/2022/08/researchers-uncover-kimusky-infra.

 ***** story: "As Nasty as Dirty Pipe" — 8 Year Old Linux Kernel Vulnerability Uncovered *****
Website no: 46 link: https://thehackernews.com/2022/08/cisa-adds-7-new-actively-exploited.html
 ***** story: CISA Adds 7 New Actively Exploited Vulnerabilities to Catalog *****
Website no: 47 link: https://thehackernews.com/2022/08/apple-releases-security-updates-to.html
 ***** story: Apple Releases Security Updates to Patch Two New Zero-Day Vulnerabilities *****
Website no: 48 link: https://thehackernews.com/2022/08/pic-and-squip-vulnerabilities-found-in.html
 ***** story: ÆPIC and SQUIP Vulnerabilities Found in Intel and AMD Processors *****
Website no: 49 link: https://thehackernews.com/2022/08/researchers-uncover-uefi-secure-boot.html
 ***** story: Researchers Uncover UEFI Secure Boot Bypass in 3 Microsoft Signed Boot Loaders *****
Website no: 50 link: https://thehackernews.com/2022/08/cisco-patches-high-severity.html
 ***** story: Cisco Patches High-Severity Vulnerability Affecting ASA a

C:\Users\moses\AppData\Local\Temp\ipykernel_5852\276599378.py:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  con_df_1.drop(columns = ['extra links'], inplace = True, axis = 1)
C:\Users\moses\AppData\Local\Temp\ipykernel_5852\276599378.py:54: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  con_df_2.drop(columns = ['full story'], inplace = True, axis = 1)


,topics,full story,extra links
0,Hackers Breach LastPass Developer System to St...,Password management service LastPass confirmed...,https://blog.lastpass.com/2022/08/notice-of-re...
1,Cybercrime Groups Increasingly Adopting Sliver...,"Nation-state threat actors are""Given Cobalt St...",https://thehackernews.com/2021/05/top-11-secur...
2,Cybercrime Groups Increasingly Adopting Sliver...,"Nation-state threat actors are""Given Cobalt St...",https://thehackernews.com/2021/05/top-11-secur...
3,Okta Hackers Behind Twilio and Cloudflare Brea...,The threat actor behind the attacks onThe acti...,https://thehackernews.com/2022/08/twilio-suffe...
4,Microsoft Uncovers New Post-Compromise Malware...,The threat actor behind the SolarWinds supply ...,https://thehackernews.com/2021/01/heres-how-so...
...,...,...,...
84,Russian State Hackers Continue to Attack Ukrai...,Russian state-sponsored actors are continuing ...,https://symantec-enterprise-blogs.security.com...
85,SOVA Android Banking Trojan Returns With New C...,The SOVA Android banking trojan is continuing ...,https://thehackernews.com/2021/09/sova-new-and...
86,Chinese Hackers Backdoored MiMi Chat App to Ta...,A pair of reports from cybersecurity firmsInfe...,https://blog.sekoia.io/luckymouse-uses-a-backd...
87,Chinese Hackers Backdoored MiMi Chat App to Ta...,A pair of reports from cybersecurity firmsInfe...,https://blog.sekoia.io/luckymouse-uses-a-backd...


In [12]:
data = pd.merge(overal_df, con_df, how = 'left',on = 'topics')
print(f"The rows before droping the duplicates {data.shape[0]}")
data.drop_duplicates(subset = ['topics', 'dates', 'author', 'links'], inplace = True)
print(f"The rows after droping the duplicates {data.shape[0]}")
data.reset_index(drop = True, inplace = True)
data

The rows before droping the duplicates 109
The rows after droping the duplicates 74


,topics,dates,author,links,truncated content,page,full story,extra links
0,Hackers Breach LastPass Developer System to St...,"August 26, 2022",Ravie Lakshmanan,https://thehackernews.com/2022/08/hackers-brea...,Password management service LastPass confirme...,main page,Password management service LastPass confirmed...,https://blog.lastpass.com/2022/08/notice-of-re...
1,Cybercrime Groups Increasingly Adopting Sliver...,"August 26, 2022",Ravie Lakshmanan,https://thehackernews.com/2022/08/cybercrime-g...,Nation-state threat actors are increasingly ...,main page,"Nation-state threat actors are""Given Cobalt St...",https://thehackernews.com/2021/05/top-11-secur...
2,Okta Hackers Behind Twilio and Cloudflare Brea...,"August 25, 2022",Ravie Lakshmanan,https://thehackernews.com/2022/08/okta-hackers...,The threat actor behind the attacks on Twili...,main page,The threat actor behind the attacks onThe acti...,https://thehackernews.com/2022/08/twilio-suffe...
3,Microsoft Uncovers New Post-Compromise Malware...,"August 25, 2022",Ravie Lakshmanan,https://thehackernews.com/2022/08/microsoft-un...,The threat actor behind the SolarWinds supply...,main page,The threat actor behind the SolarWinds supply ...,https://thehackernews.com/2021/01/heres-how-so...
4,U.S. Government Spending Billions on Cybersecu...,"August 25, 2022",The Hacker News,https://thehackernews.com/2022/08/us-governmen...,"In recent months, the House of Representative...",main page,"In recent months, the House of Representatives...",https://rollcall.com/2022/07/12/house-appropri...
...,...,...,...,...,...,...,...,...
69,Malicious Browser Extensions Targeted Over a M...,"August 17, 2022",Ravie Lakshmanan,https://thehackernews.com/2022/08/malicious-br...,More than 1.31 million users attempted to ins...,Malware,More than 1.31 million users attempted to inst...,https://securelist.com/threat-in-your-browser-...
70,North Korea Hackers Spotted Targeting Job Seek...,"August 17, 2022",Ravie Lakshmanan,https://thehackernews.com/2022/08/north-korea-...,The North Korea-backed Lazarus Group has been...,Malware,The North Korea-backed Lazarus Group has been ...,https://thehackernews.com/2020/06/military-aer...
71,Russian State Hackers Continue to Attack Ukrai...,"August 16, 2022",Ravie Lakshmanan,https://thehackernews.com/2022/08/russian-stat...,Russian state-sponsored actors are continuing...,Malware,Russian state-sponsored actors are continuing ...,https://symantec-enterprise-blogs.security.com...
72,SOVA Android Banking Trojan Returns With New C...,"August 15, 2022",Ravie Lakshmanan,https://thehackernews.com/2022/08/sova-android...,The SOVA Android banking trojan is continuing...,Malware,The SOVA Android banking trojan is continuing ...,https://thehackernews.com/2021/09/sova-new-and...


In [13]:
# #create a content df 
# con_df = pd.DataFrame()

# for i in content_soup.find_all('div', {'class':'main-box clear'}):
    
#     #get the header
#     hd = i.find_all('h1')
#     header = [x.text.strip() for x in hd][0]
    
#     #get the url 
#     con_ls_df = {"content":[]}
#     for p in i.find_all('p'):
#         con_ls = [x.text.strip() for x in p][0]
#         con_ls_df['content'].append(con_ls)
# #         print(con_ls)
    
#     #get con frames 
#     con_frames = {'topic':header, 'full story':con_ls_df}
#     con_frames_df = pd.DataFrame(con_frames)
#     con_df = pd.concat([con_df, con_frames_df])
    
# # reset index 
# con_df.reset_index(drop = True, inplace = True)
# con_df

In [14]:
# type(con_ls_df)

In [15]:
# con_ls_df

In [16]:
#generate a filename 
name = f"hacker_news_website"

#get todays date
today = datetime.now()

# format the dates
formated_date = today.strftime("%d_%b").lower()

#extension
extension = ".xlsx"

#generate the name
new_filename = name+"_"+formated_date
old_filename = name+"_"+formated_date+"_"+"oldfile"
print(old_filename ,"\n", new_filename)

hacker_news_website_26_aug_oldfile 
 hacker_news_website_26_aug


In [17]:
#create a copy
df = data.copy()

In [18]:
#create the connection 
# table_nam
db_host = 'research-dev-db.cluster-c30rc84jwwhk.us-east-2.rds.amazonaws.com'
# db_host = '192.168.8.221'
db_port = 5432
db_password = 'researchpassword'
db = 'postgres'
db_user = 'postgres'


def psql_insert_copy(table, conn, keys, data_iter):
    # gets a DBAPI connection that can provide a cursor
    dbapi_conn = conn.connection
    with dbapi_conn.cursor() as cur:
        s_buf = StringIO()
        writer = csv.writer(s_buf)
        writer.writerows(data_iter)
        s_buf.seek(0)

        columns = ', '.join('"{}"'.format(k) for k in keys)
        if table.schema:
            table_name = '{}.{}'.format(table.schema, table.name)
        else:
            table_name = table.name

        sql = 'COPY {} ({}) FROM STDIN WITH CSV'.format(
            table_name, columns)
        cur.copy_expert(sql=sql, file=s_buf)

engine = create_engine(f'postgresql://{db_user}:{db_password}@{db_host}:{db_port}/{db}')

#write the data if the table exists , replace the data
try:
    data.to_sql(new_filename, engine, method=psql_insert_copy, if_exists='replace')
except Exception as e:
    print(f'The error is: {str(e)}')
    pass

In [19]:
#rename the file if is in directory 
try:
  os.rename(old_filename+'.xlsx', new_filename+'xlsx')
except Exception as e:
  print(f"The error is: {str(e)}")
  pass

The error is: [WinError 2] The system cannot find the file specified: 'hacker_news_website_26_aug_oldfile.xlsx' -> 'hacker_news_website_26_augxlsx'


In [20]:
#write the file
try:
    data.to_excel(new_filename+'.xlsx', sheet_name = 'tweets data', index = False)
#     coordinates_df.to_excel("african cities coordinates"+'.xlsx', sheet_name = 'tweets data', index = False)
except BaseException as e:
    print('-'*20)
    print(f"The error is {str(e)}")
    pass

In [ ]:
//*[@id="Blog1_blog-pager-older-link"]
//*[@id="Blog1_blog-pager-older-link"]
//*[@id="Blog1_blog-pager-older-link"]